In [1]:
import torch
#导入
from datasets import Dataset
from numpy.core.numeric import True_
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer


bin D:\anaconda\ana\envs\VectorEnv\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
#导入数据集
ds = Dataset.load_from_disk("./alpaca_data_zh")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
#数据预处理
tokenizer = AutoTokenizer.from_pretrained("D:\Hugging Face Hub\Llama\modelscope\Llama-2-7b-ms")
#tokenizer.padding_size = "right" batch为2的时候需要调整(Llama)
#tokenizer.pad_token_id = 2

In [4]:
def process_func(example):
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ",add_special_tokens=False)
    response = tokenizer(example["output"] ,add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"]+[tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }
    #数据预处理注意tokenizer会被切开

In [5]:
tokenizer_ds = ds.map(process_func,remove_columns=ds.column_names)

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

In [12]:
#创建模型
import torch
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained("D:\Hugging Face Hub\Llama\modelscope\Llama-2-7b-ms",low_cpu_mem_usage=True,torch_dtype=torch.half,device_map = "auto",quantization_config=quantization_config)

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [7]:
#model.enable_input_require_grads()

In [17]:
#Lora
from peft import  LoraConfig,get_peft_model,TaskType
config = LoraConfig(task_type=TaskType.CAUSAL_LM)
model = get_peft_model(model,config)
model = model.half()#更改Lore的float

In [18]:
#配置训练参数
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,#2的话涉及到一个padding的问题
    #gradient_accumulation_steps=2,
    logging_steps=100,
    num_train_epochs=1  #gradient_checkpointing通过牺牲少量训练速度来显著减少模型训练时的显存占用。
    #adam_epsilon=1e-4
)

In [23]:
#配置训练器
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenizer_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

C:\Users\wangyuncan\AppData\Local\Temp\ipykernel_11572\2173757612.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [24]:
#训练
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 5.21 GiB is allocated by PyTorch, and 32.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [35]:
#预测
from transformers import pipeline
pipe = pipeline("text-generation",model = model,tokenizer=tokenizer,device=0)

Device set to use cuda:0
